In [1]:
#r "nuget: FSharp.Stats, 0.5.1-preview.1"
#r "nuget: Plotly.NET.Interactive, 4.2.1"

open FSharp.Stats
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open FSharp.Data
open System

// use a script to import data in all notebooks > 1
#load "import.fsx"
open Import
let orders = Import.orders

let getDepartmentColor (department: string) = 
    match department with 
    | "Breakroom Bandits" -> "#2b3ae9"
    | "Genesis" -> "#f7da41"
    | "We Tried" -> "#008b66"
    | "No Lucks Given" -> "#987200"
    | "Toon Squad" -> "#ff7f0e"
    | "Rumor Spreaders" -> "#20b2aa"
    | "Risky Biscuits" -> "#a230ed"
    | "Recruitables" -> "#d21102"
    | "Employees of the Moment" -> "#19d3f3"
    | "Chargers" -> "#dea57b"
    | "Kickstarters" -> "#dea57b"
    | _ -> "#8b8b8b"

Installed Packages FSharp.Stats, 0.5.1-preview.1 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\schne\.nuget\packages\plotly.net.interactive\4.2.1\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

Lets isolate the beverage orders from a person of your choice. Count how many beverages where logged each day and visualize the data as column chart.

Hint: Use `(date: System.DateTime).ToToShortDateString()`

In [2]:
let beverageOrders = 
    orders
    |> Array.filter (fun x -> x.Name = "Benedikt V." && x.Category = Beverage)
    |> Array.countBy (fun x -> 
        x.DateTime.ToShortDateString()
        )
    |> Chart.Column
beverageOrders

<!-- Plotly chart will be drawn inside this DIV -->

As you can see there is no day of zero count. But obviously nobody is there every single day since 2018. Plotly does not visualize days that are not present in the input data. 


In [22]:
open Plotly.NET.LayoutObjects

let paddedBeverageCounts =
    let rec loop (date) (acc: (string*int) list) = 
        if date >= System.DateTime.Now then 
            acc |> List.rev
        else 
            let orders = 
                orders
                |> Array.filter (fun x -> x.Name = "Benedikt V." && x.Category = Beverage && x.DateTime.ToShortDateString() = date.ToShortDateString())
                |> Array.length
            loop (date.AddDays 1) ((date.ToShortDateString(),orders)::acc)
    loop (System.DateTime(2018,08,23)) []


let paddedBeerCounts =
    let rec loop (date) (acc: (string*int) list) = 
        if date >= System.DateTime.Now then 
            acc |> List.rev
        else 
            let orders = 
                orders
                |> Array.filter (fun x -> x.Name = "Benedikt V." && x.Category = Beer && x.DateTime.ToShortDateString() = date.ToShortDateString())
                |> Array.length
            loop (date.AddDays 1) ((date.ToShortDateString(),orders)::acc)
    loop (System.DateTime(2018,08,23)) []


let paddedCoffeeCounts =
    let rec loop (date) (acc: (string*int) list) = 
        if date >= System.DateTime.Now then 
            acc |> List.rev
        else 
            let orders = 
                orders
                |> Array.filter (fun x -> x.Name = "Benedikt V." && x.Category = Coffee && x.DateTime.ToShortDateString() = date.ToShortDateString())
                |> Array.length
            loop (date.AddDays 1) ((date.ToShortDateString(),orders)::acc)
    loop (System.DateTime(2018,08,23)) []



[
    paddedCoffeeCounts |> Chart.StackedColumn  |> Chart.withTraceInfo "Coffee"
    paddedBeverageCounts |> Chart.StackedColumn |> Chart.withTraceInfo "Beverage"
    paddedBeerCounts |> Chart.StackedColumn |> Chart.withTraceInfo "Beer"
]
|> Chart.combine
|> Chart.withTemplate ChartTemplates.lightMirrored
|> Chart.withYAxisStyle( "order count")
|> Chart.withSize(1000,600)
|> Chart.withXAxis(
    LinearAxis.init(
        TickLabelStep = 10
    )
)


<!-- Plotly chart will be drawn inside this DIV -->

The following cell creates a function that takes a string option that defines a user name (if the option type is None, all users are selected) and two dates, that define the beginning and end of the data assessment for visualize the daily drinking behaviour for every day of a week.
(dont delete) 

In [4]:

let plotWeekDistByNameAndDay (name: string option) (fromDate: System.DateTime option) (toDate: System.DateTime option) =

    let myAxisRange title (min,max) showtick = LinearAxis.init(Title=Title.init title,Range=StyleParam.Range.MinMax (min,max),Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=true,ShowLine=true,ShowTickLabels=showtick)

    let nameSeq =
        if name.IsNone then 
            if fromDate.IsSome then orders |> Array.filter (fun x -> x.DateTime > fromDate.Value && x.DateTime < toDate.Value) else orders
            |> Seq.groupBy (fun x -> x.DateTime.DayOfWeek)
            |> Seq.sortBy (fun (key,items) -> key)
            |> fun x -> Seq.append (Seq.tail x) (seq [Seq.head x])
            //|> Seq.map snd
        else 
            if fromDate.IsSome then orders |> Array.filter (fun x -> x.DateTime > fromDate.Value && x.DateTime < toDate.Value) else orders
            |> Seq.filter (fun x -> x.Name = name.Value)
            |> Seq.groupBy (fun x -> x.DateTime.DayOfWeek)
            |> Seq.sortBy (fun (key,items) -> key)
            |> fun x -> Seq.append (Seq.tail x) (seq [Seq.head x])
            //|> Seq.map snd

    let coffee day (seq: seq<Order>) showtick =
        seq
        |> Seq.filter (fun x -> x.Category = Category.Coffee)
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                                                         
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#4b77ad") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "coffee" else "") (-4000000.,83000000.) showtick) |> Chart.withTraceInfo (sprintf "%s coffee" day))  
    let beverage day (seq: seq<Order>) showtick=                                                                                                                                                                                          
        seq                                                                                                                                                                                                                 
        |> Seq.filter (fun x-> x.Category = Category.Beverage)                                                                                                                                                                     
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#ad504b") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "beverage" else "") (-4000000.,83000000.) showtick) |> Chart.withTraceInfo (sprintf "%s beverage" day))
    let beer day (seq: seq<Order>) showtick=                                                                                                                                                                                              
        seq                                                                                                                                                                                                                 
        |> Seq.filter (fun x-> x.Category = Category.Beer)                                                         
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#4bad81") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "beer" else "") (-4000000.,83000000.) showtick) |> Chart.withTraceInfo (sprintf "%s beer" day))   
    let other day (seq: seq<Order>) showtick=                                                                                                                                                                                              
        seq                                                                                                                                                                                                                 
        |> Seq.filter (fun x-> x.Category <> Beverage && x.Category <> Coffee && x.Category <> Deposit && x.Category <> Debit && x.Category <> TestStuff)
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#999999") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "Milk/Food/other" else "") (-4000000.,83000000.) showtick)  |> Chart.withTraceInfo (sprintf "%s other" day))   
            
    nameSeq
    |> Seq.mapi (fun i (day,x) -> 
        let showtick = i=6 
        coffee (day.ToString()) x showtick ,
        beverage (day.ToString()) x showtick ,
        beer (day.ToString()) x showtick )
    |> Array.ofSeq
    |> Array.unzip3
    |> fun (cof,bev,bee) ->
        //let showPlot category cat =
        //    category
            //|> Chart.Stack 1
            //|> Chart.withSize(900.,900.) 
            //|> Chart.withX_Axis (axis (sprintf "DailyDistribution %s - %s" cat name))
        [|
            cof 
            bev 
            bee 
        |]
        |> JaggedArray.transpose
        |> Array.concat
        |> Chart.Grid(7,3)
        |> Chart.withSize (1200.,800.)
        |> Chart.withTitle (sprintf "daily order distribution for %s" (if name.IsNone then "all" else name.Value))
        |> Chart.withMarginSize(Left=50.,Bottom=50.,Top=50.)
        |> fun l -> 
            l 
            |> Chart.withDescription [Giraffe.ViewEngine.HtmlElements.rawText "xAxis=Time, yAxis=count; blue=coffee, red=beverages, green=beer; top=Monday, bottom=Sunday"]

let chartBenediktAllTime = plotWeekDistByNameAndDay (Some "Benedikt V.") (None) (None)
let chartBenediktLockDown = plotWeekDistByNameAndDay (Some "Benedikt V.") (Some (System.DateTime(2020,04,01))) (Some (System.DateTime(2020,07,01)))


let chartAllAlltime = plotWeekDistByNameAndDay (None) None None
let chartAllUntilLockdown = plotWeekDistByNameAndDay (None) (Some (System.DateTime(2018,04,01))) (Some (System.DateTime(2020,04,01)))
let chartAllSinceLockdown = plotWeekDistByNameAndDay (None) (Some (System.DateTime(2020,04,01))) (Some (System.DateTime(2020,07,01)))

chartBenediktAllTime

<!-- Plotly chart will be drawn inside this DIV --> xAxis=Time, yAxis=count; blue=coffee, red=beverages, green=beer; top=Monday, bottom=Sunday

Create a histogram that shows all ticks as daytime projected on a single day.

Hint: You can set the day of all orders to 01/01/1970, wich in plotly ticks translates to (-4000000.,83000000.)

In [5]:
let dailyDist =  
    let myAxisRange title (min,max) = LinearAxis.init(Title=Title.init title,Range=StyleParam.Range.MinMax (min,max),Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=true,ShowLine=true,ShowTickLabels=true)
    let before5am = orders |> Array.filter (fun x -> x.DateTime.TimeOfDay < System.DateTime(2020,1,1,5,0,0).TimeOfDay)
    orders
    |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                                                         
    |> fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#4b77ad") 
    |> Chart.withXAxis (myAxisRange ("") (-4000000.,83000000.)) 
    |> Chart.withYAxisStyle("#count") 
    |> Chart.withDescription [Giraffe.ViewEngine.HtmlElements.rawText (sprintf "Ticks total: %i<br>Ticks before 5am: %i (%.2f %%)" orders.Length before5am.Length (100. * float before5am.Length/float orders.Length))]
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withTitle "daily ticking distribution"

dailyDist

<!-- Plotly chart will be drawn inside this DIV -->

Check with for every user which time span was between the first and last drink. Visualize as box plot.

In [6]:
orders 
|> Array.filter (fun x -> x.DateTime.TimeOfDay > (DateTime(2023,1,1,5,0,0)).TimeOfDay)
|> Array.groupBy (fun x -> x.Name)
|> Array.map (fun (name,orders) -> 
    let tmp = 
        orders
        |> Array.groupBy (fun o -> o.DateTime.Date)
        |> Array.choose (fun (date,items) -> 
            let min = items |> Array.map(fun x -> x.DateTime) |> Array.min
            let max = items |> Array.map(fun x -> x.DateTime) |> Array.max 
            if items.Length < 3 || (max - min).Hours = 0 then 
                None 
            else
            Some (float (max - min).Hours + float (max - min).Minutes / 60.)
        )
        |> fun x -> (Chart.BoxPlot(Y=x,Name=name))
    tmp
    )
|> Chart.combine
|> Chart.withTemplate ChartTemplates.lightMirrored 
|> Chart.withYAxisStyle "time span between first and last order (hours)"

<!-- Plotly chart will be drawn inside this DIV -->

some more functions to vizualize personal properties (dont delete)

In [10]:


orders 
|> Array.groupBy (fun x -> x.Name)
|> Array.map (fun (name,orders) -> 
    let min = orders |> Array.map(fun x -> x.DateTime) |> Array.min
    let max = orders |> Array.map(fun x -> x.DateTime) |> Array.max 
    let employmentDuration = (max - min).Days
    let lg,col = 
        if System.DateTime.Now.AddDays(-60.) < max then 
            if orders.[0].DateTime.AddDays(60.) > min then 
                "total time employed",Color.fromString "grey" 
            else
                "currently employed",Color.fromString "blue" 
        else 
            if orders.[0].DateTime.AddDays(30.) > min then 
                "from start, not now",Color.fromString "green" 
            else
                "valid time span",Color.fromString "orange"
    employmentDuration,Chart.Column([name,float employmentDuration / 365.],MarkerColor=col) |> Chart.withTraceInfo(LegendGroup=lg,LegendGroupTitle=Title.init lg)
    )
|> Array.sortByDescending fst
|> Array.map snd
|> Chart.combine
|> Chart.withTemplate ChartTemplates.lightMirrored
|> Chart.withYAxisStyle "employment duration (years)"
|> Chart.withTitle "employment duration"
|> Chart.withDescription [(Giraffe.ViewEngine.HtmlElements.rawText """<br>grey: total time employed<br>blue: currently employed<br>green: from start, not now<br>orange: valid time span""")]

<!-- Plotly chart will be drawn inside this DIV -->

In [11]:


orders 
|> Array.filter (fun x -> x.DateTime.TimeOfDay < (DateTime(2023,1,1,12,0,0)).TimeOfDay)
|> Array.groupBy (fun x -> x.Name)
|> Array.map (fun (name,orders) -> 
    let tmp = 
        orders
        |> Array.groupBy (fun o -> o.DateTime.Date)
        |> Array.choose (fun (date,items) -> 
            let min = items |> Array.filter (fun t -> t.Category = Category.Coffee) 
            if min = [||] then 
                None 
            else
                let tmp = min |> Array.minBy (fun x -> x.DateTime)
                Some (System.DateTime(1900,01,01,tmp.DateTime.TimeOfDay.Hours,tmp.DateTime.TimeOfDay.Minutes,tmp.DateTime.TimeOfDay.Seconds))
        )
    name,tmp
    )
|> Array.map (fun (name,x) -> Chart.BoxPlot(Y=x,Name=name))
|> Chart.combine
|> Chart.withTemplate ChartTemplates.lightMirrored
|> Chart.withTitle "daily first tick distribution"
|> Chart.withYAxisStyle "first tick"


<!-- Plotly chart will be drawn inside this DIV -->